In [67]:
import pandas as pd
import numpy as np
import glob
from Solver import solve

In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
num_bids = 14
num_goods = 11
seed = 2024
np.random.seed(seed)

In [70]:
def unify_list(lis):
    # print(lis)
    ret_lis = []
    for i in range(num_goods):
        if i in lis:
            ret_lis.append(1)
        else:
            ret_lis.append(0)
    return ret_lis

In [71]:
folder_path = 'ressources'
file_pattern = '*.txt'
file_paths = glob.glob(f'{folder_path}/{file_pattern}')

columns = []
for i in range(num_bids):
    columns.append('Price_' + str(i))
    for j in range(num_goods):
        columns.append('Bid_' + str(i) + '_' + str(j))
        
for i in range(num_bids):
    columns.append('Solution_' + str(i))

df = pd.DataFrame(columns=columns)

for file_path in file_paths:
    with open(file_path, 'r') as file:
        lines = file.readlines()

    bids = np.zeros([num_bids, num_goods + 1])

    file_number = int(file_path[11:15])
    bid_number = 0
    for line in lines:
        if line.startswith('%') or line.startswith('\n') or line.startswith('goods') or line.startswith(
                'bids') or line.startswith('dummy'):
            continue
        else:
            _, price, *goods, _ = line.split("\t")
            
            bids[bid_number, 0] = price
            
            goods = list(map(int, goods))
            for good in goods:
                if good >= num_goods:
                    print("Error: good is " + str(good))
                    break
                bids[bid_number, good + 1] = 1
            
            bid_number += 1

    price_sum = np.sum(bids[:, 0])
    bids[:, 0] = bids[:, 0] / price_sum
    solutions = solve(bids, num_bids=num_bids, num_goods=num_goods)
    
    row = np.concatenate((bids.flatten(), solutions))
    df.loc[file_number] = row

In [72]:
df.head()

,Price_0,Bid_0_0,Bid_0_1,Bid_0_2,Bid_0_3,Bid_0_4,Bid_0_5,Bid_0_6,Bid_0_7,Bid_0_8,...,Solution_4,Solution_5,Solution_6,Solution_7,Solution_8,Solution_9,Solution_10,Solution_11,Solution_12,Solution_13
365,0.097675,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
376,0.117325,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270,0.086037,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
653,0.100991,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,0.116555,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [73]:
df.shape

(1000, 182)

In [74]:
import os

try:
    # Write the DataFrame to a CSV file in the 'data' directory
    df.to_csv('data/computed_values.csv', index=False)
    print("File 'computed_values.csv' written successfully in directory:", os.getcwd() + "/data")
except Exception as e:
    print("An error occurred while writing the file:", e)
    
# df.sample(10)


File 'computed_values.csv' written successfully in directory: /home/kassian/Alles/Uni/Master/Semester-2/IDP/machine-learning-for-auctions/data
